In [12]:
import pandas as pd
import numpy as np
from functools import reduce

In [13]:
# metrics dataframes
df_emissions = pd.read_csv('co2_emissions.csv')
df_air_pollution = pd.read_csv('air_pollution.csv')
df_exports = pd.read_csv('exports.csv')
df_forest_area = pd.read_csv('forest_area.csv')
df_fossil_fuels = pd.read_csv('fossil_fuels.csv')
df_GDP = pd.read_csv('GDP.csv')
df_imports = pd.read_csv('imports.csv')
df_livestock_production = pd.read_csv('livestock_production.csv')
df_renewable_energy = pd.read_csv('renewable_energy.csv')

# additonal datasets
df_income = pd.read_csv('income.csv')
df_country_codes = pd.read_excel('country_codes.xlsx')

In [14]:
def filter_convert_long(df,new_column_name, start_year=1990, end_year=2020):
    year_columns = [col for col in df.columns if col.isdigit() and start_year <= int(col) <= end_year]
    extra_columns = ['Country Name', 'Country Code']
    columns_to_keep = [col for col in extra_columns if col in df.columns] + year_columns
    filtered_df = df[columns_to_keep]
    
    df_long = pd.melt(filtered_df, id_vars=['Country Name', 'Country Code'], var_name='Year', value_name=new_column_name)
    df_long['Year'] = df_long['Year'].astype(int)
    
    return df_long

In [15]:
# filtering to retain data from 1990 - 2020 only
# converting dataframes to long format

df_emissions = filter_convert_long(df_emissions, new_column_name='CO2_Emissions')
df_air_pollution = filter_convert_long(df_air_pollution, new_column_name='Air_Pollution')
df_exports = filter_convert_long(df_exports, new_column_name='Exports')
df_forest_area = filter_convert_long(df_forest_area, new_column_name='Forest_Area')
df_fossil_fuels = filter_convert_long(df_fossil_fuels, new_column_name='Fossil_Fuel_Energy')
df_GDP = filter_convert_long(df_GDP, new_column_name='GDP')
df_imports = filter_convert_long(df_imports, new_column_name='Imports')
df_livestock_production = filter_convert_long(df_livestock_production, new_column_name='Livestock_Production')
df_renewable_energy = filter_convert_long(df_renewable_energy, new_column_name='Renewable_Energy')

In [16]:
df_air_pollution = pd.merge(df_country_codes, df_air_pollution, on='Country Code')


dfs = [df_air_pollution, df_exports, df_forest_area, df_fossil_fuels,
       df_GDP, df_imports, df_livestock_production, df_renewable_energy]

try:
    merged_df = reduce(lambda left, right: pd.merge(left, right, on=['Country Code', 'Year'], suffixes=('', '_df2')), dfs)
    merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_df2')]
    merged_row_count = merged_df.shape[0]
except Exception as e:
    merged_row_count = f"Error: {e}"

In [17]:
merged_df

,Country Code,Country Name,Year,Air_Pollution,Exports,Forest_Area,Fossil_Fuel_Energy,GDP,Imports,Livestock_Production,Renewable_Energy
0,AFG,Afghanistan,1990,64.174097,NaN,1.852782,NaN,NaN,NaN,67.11,23.0
1,AFG,Afghanistan,1991,64.188153,NaN,1.852782,NaN,NaN,NaN,68.55,23.7
2,AFG,Afghanistan,1992,64.210526,NaN,1.852782,NaN,NaN,NaN,68.15,27.4
3,AFG,Afghanistan,1993,64.244141,NaN,1.852782,NaN,NaN,NaN,72.09,28.5
4,AFG,Afghanistan,1994,64.291923,NaN,1.852782,NaN,NaN,NaN,79.06,30.1
...,...,...,...,...,...,...,...,...,...,...,...
5978,ZWE,Zimbabwe,2016,22.570698,4.098132e+09,45.570273,NaN,2.054868e+10,6.426700e+09,103.85,81.7
5979,ZWE,Zimbabwe,2017,17.997754,3.456997e+09,45.451183,NaN,1.758489e+10,5.340673e+09,101.98,82.0
5980,ZWE,Zimbabwe,2018,19.112099,8.936585e+09,45.332093,NaN,3.415607e+10,9.695644e+09,102.21,79.7
5981,ZWE,Zimbabwe,2019,18.528607,6.015688e+09,45.213002,NaN,2.183223e+10,6.165817e+09,106.68,81.0


In [18]:
merged_df.to_csv('emission_analysis.csv', index=False)